# TODO List

1. combine into SpectraViz class
1. continuum normalization
1. implement BNN and label uncertanties
1. subset (lasso) select (widgets.SelectMultiple)
1. layout
1. create Python package
1. scale to Big Data

In [1]:
%matplotlib widget

import h5py
import ipywidgets as widgets
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn import decomposition
from sklearn import manifold
import umap

import plot

matplotlib.__version__

'3.3.2'

In [2]:
SIZE = 2048

with h5py.File("data/dataset.hdf5", "r") as datafile:
    ids = datafile["id_tr"][:]
    rnd_idx = np.random.randint(ids.shape[0], size=SIZE)
    ids = ids[rnd_idx]
    X = datafile["X_tr"][:][rnd_idx]
    z = datafile["z_tr"][:][rnd_idx].reshape(-1, 1)
ids.shape, X.shape, z.shape

((2048,), (2048, 512), (2048, 1))

In [3]:
fig, ax = plt.subplots()
plot.spectrum(ax, X[0], label="z = " + str(z[0, 0]))
plot.redshift(ax, z[0, 0])
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
w = widgets.SelectMultiple(
    options=ids[:16],
    rows=10
)
w

/home/podondra/astro/spectraviz/venv/lib/python3.8/site-packages/traitlets/traitlets.py:586: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  silent = bool(old_value == new_value)


SelectMultiple(options=((8848, 57875, 125), (3793, 55214, 801), (4778, 55706, 848), (7340, 56726, 125), (7339,…

In [5]:
w.options = ids[16:24]

In [6]:
w.value

()

In [7]:
def run_pca(whiten=False, svd_solver="auto"):
    reducer = decomposition.PCA(n_components=2, whiten=whiten, svd_solver=svd_solver)
    embedding = reduce_dim(reducer)
    plot_projection(embedding)

pca_widget = widgets.interactive(
    run_pca, {"manual": True},
    svd_solver=["auto", "full", "arpack", "randomized"])

def run_tsne(
        perplexity=30, early_exaggeration=12.0, learning_rate=200.0,
        n_iter=1000, init="random", method="barnes_hut", n_jobs=-1):
    reducer = manifold.TSNE(
        n_components=2, perplexity=perplexity,
        early_exaggeration=early_exaggeration, learning_rate=learning_rate,
        n_iter=n_iter, init=init, verbose=2, method=method, n_jobs=n_jobs)
    embedding = reduce_dim(reducer)
    plot_projection(embedding)

tsne_widget = widgets.interactive(
    run_tsne, {"manual": True},
    perplexity=(5, 50, 1),
    early_exaggeration=widgets.FloatText(value=12.0, step=0.1),
    learning_rate=widgets.FloatText(value=200.0, step=0.1),
    n_iter=widgets.IntText(1000),
    init=["random", "pca"],
    method=["barnes_hut", "exact"],
    n_jobs=widgets.IntText(value=-1))

def run_umap(
        n_neighbors=15, metric="euclidean", n_epochs=None, learning_rate=1.0,
        init="spectral", min_dist=0.1, low_memory=False):
    reducer = umap.UMAP(
        n_neighbors=n_neighbors, metric=metric, n_epochs=n_epochs,
        learning_rate=learning_rate, init=init, min_dist=min_dist,
        low_memory=low_memory, verbose=True)
    embedding = reduce_dim(reducer)
    plot_projection(embedding)

umap_widget = widgets.interactive(
    run_umap, {"manual": True},
    n_neighbors=(2, 100, 1),
    metric=["euclidean", "manhattan", "chebyshev", "minkowski"],
    n_epochs=widgets.fixed(None),
    learning_rate=widgets.FloatText(value=1.0, min=0, step=0.1),
    init=["spectral", "random"],
    min_dist=widgets.FloatText(value=0.1, min=0, step=0.1))

reducer_out = widgets.Output(layout={'border': '1px solid black'})

@reducer_out.capture(clear_output=True)
def reduce_dim(reducer):
    return reducer.fit_transform(X)

def plot_projection(embedding):
    proj_ax.clear()
    collection = proj_ax.scatter(embedding[:, 0], embedding[:, 1], c=z, picker=True)
    proj_fig.colorbar(collection, ax=proj_ax)

titles = ['PCA', 't-SNE', 'UMAP']
childrens = [pca_widget, tsne_widget, umap_widget]
tabs = widgets.Tab()
tabs.children = childrens
for i, title in enumerate(titles):
    tabs.set_title(i, title)

proj_out = widgets.Output()
with proj_out:
    proj_fig, proj_ax = plt.subplots(constrained_layout=True)

def onpick(event):
    spec_ax.clear()
    plot.spectrum(spec_ax, X[event.ind[0]])
    spec_ax.grid(True)

proj_fig.canvas.mpl_connect("pick_event", onpick)

spec_out = widgets.Output()
with spec_out:
    spec_fig, spec_ax = plt.subplots(constrained_layout=True)

layout = widgets.Layout(align_items="stretch", flex_flow="row wrap")
widgets.HBox(children=[tabs, reducer_out, proj_out, spec_out], layout=layout)